In [1]:
import pandas as pd
import re

df = pd.read_csv(
    'club-games.csv',
    parse_dates=['fecha'],
    date_parser=lambda x: pd.to_datetime(x, format='%Y-%m-%d'),
    dtype={'hora': str}
)
df['hora'] = df['hora'].str.strip()


C:\Users\MAM\AppData\Local\Temp\ipykernel_10324\2723440021.py:4: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv(


### Preprocesamientos

In [2]:
df.head()

,partida_id,fecha,hora,juego,coste_juego,copia_juego,expansiones,jugadores,personal,resultado
0,1,2021-08-24,14:44,Scythe,60€,10,"Invaders from Afar (coste: 30€, copia: 4)",Melisa de Barrera; Toño Arroyo Vilalta; María ...,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35€,8,NaN,Nieves Zapata Cáceres; Matilde Palomares Varga...,Joel Ángel Campillo,René Valentín Catalá ganó
2,3,2021-04-06,14:46,Catan,45€,8,"Ciudades y Caballeros (coste: 30€, copia: 9); ...",Itziar Martín Tomas; Elisa Franch Abad; Harold...,Julio Luján Valderrama,Haroldo Saez-Blanca ganó
3,4,2020-10-02,02:20,Agricola,55€,2,NaN,Flora Díaz Soler; Juan Manuel Oller Palomo; Es...,Amado del Piñeiro,Juan Manuel Oller Palomo ganó
4,5,2024-01-30,14:09,Arkham Horror: The Card Game,55€,4,NaN,Bernabé Bayona Requena; Adriana Cazorla Nicola...,Débora Escolano Martinez,Alonso Victor Anaya Borrell ganó


In [3]:
df.coste_juego = df.coste_juego.str.replace('€', '')
df.coste_juego.head()

0    60
1    35
2    45
3    55
4    55
Name: coste_juego, dtype: object

In [4]:
df = df.rename(columns={'partida_id': 'id_partida', 'juego': 'nombre_juego', 'jugadores': 'nombre_jugador', 'copia_juego':'id_juego','copia_expansion':'id_expansion'})
df.expansiones = df.expansiones.str.split('; ')
df = df.explode('expansiones')
df = pd.concat([df.drop(columns=['expansiones']), df.expansiones.str.extract(r'^(?P<nombre>.*) \(coste: (?P<coste>\d+)€.*, copia: (?P<id_expansion>\d+)\)$')\
    .rename(columns={'nombre': 'nombre_expansion', 'coste': 'coste_expansion'})], axis=1)
df.head(4)

,id_partida,fecha,hora,nombre_juego,coste_juego,id_juego,nombre_jugador,personal,resultado,nombre_expansion,coste_expansion,id_expansion
0,1,2021-08-24,14:44,Scythe,60,10,Melisa de Barrera; Toño Arroyo Vilalta; María ...,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35,8,Nieves Zapata Cáceres; Matilde Palomares Varga...,Joel Ángel Campillo,René Valentín Catalá ganó,NaN,NaN,NaN
2,3,2021-04-06,14:46,Catan,45,8,Itziar Martín Tomas; Elisa Franch Abad; Harold...,Julio Luján Valderrama,Haroldo Saez-Blanca ganó,Ciudades y Caballeros,30,9
2,3,2021-04-06,14:46,Catan,45,8,Itziar Martín Tomas; Elisa Franch Abad; Harold...,Julio Luján Valderrama,Haroldo Saez-Blanca ganó,Navegadores,25,8


In [5]:
df.nombre_jugador = df.nombre_jugador.str.split('; ')
df = df.explode('nombre_jugador')
df.head(10)

,id_partida,fecha,hora,nombre_juego,coste_juego,id_juego,nombre_jugador,personal,resultado,nombre_expansion,coste_expansion,id_expansion
0,1,2021-08-24,14:44,Scythe,60,10,Melisa de Barrera,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,Toño Arroyo Vilalta,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,María Del Carmen Caballero Caparrós,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,Ariadna Calvo,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,Moisés Álamo-Llano,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,Jose Ignacio del Angulo,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós ganó,Invaders from Afar,30,4
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35,8,Nieves Zapata Cáceres,Joel Ángel Campillo,René Valentín Catalá ganó,NaN,NaN,NaN
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35,8,Matilde Palomares Vargas,Joel Ángel Campillo,René Valentín Catalá ganó,NaN,NaN,NaN
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35,8,Bernabé Conde Tudela,Joel Ángel Campillo,René Valentín Catalá ganó,NaN,NaN,NaN
1,2,2023-07-02,18:43,Betrayal at House on the Hill,35,8,René Valentín Catalá,Joel Ángel Campillo,René Valentín Catalá ganó,NaN,NaN,NaN


In [6]:
df.resultado = df.resultado.str.replace(' ganó', '').str.strip()
df.head(3)

,id_partida,fecha,hora,nombre_juego,coste_juego,id_juego,nombre_jugador,personal,resultado,nombre_expansion,coste_expansion,id_expansion
0,1,2021-08-24,14:44,Scythe,60,10,Melisa de Barrera,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,Toño Arroyo Vilalta,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós,Invaders from Afar,30,4
0,1,2021-08-24,14:44,Scythe,60,10,María Del Carmen Caballero Caparrós,Alberto Torralba Sierra,María Del Carmen Caballero Caparrós,Invaders from Afar,30,4


Arreglamos este problema que luego al pasar a sql da problemas.

In [7]:
df.personal = df.personal.str.replace('Cruz de Saez', 'Cruz de Sáez')

## 2. Hacemos cada tabla
### JUGADOR

In [8]:
jugador = df.nombre_jugador.drop_duplicates().to_frame()

jugador

,nombre_jugador
0,Melisa de Barrera
0,Toño Arroyo Vilalta
0,María Del Carmen Caballero Caparrós
0,Ariadna Calvo
0,Moisés Álamo-Llano
...,...
49999,Cesar Escribano Adán
49999,Rubén Conesa Arcos
49999,Jennifer Bello Casares
49999,Jacinta Castilla Amaya


In [9]:
jugador[jugador.nombre_jugador.str.lower().str.contains('luis rosell')]

,nombre_jugador
5544,Luis Rosell
6527,Luis Roselló
15760,Edu Jose Luis Rosell Porras
33079,Juan Luis Roselló


#### JUEGO

In [10]:
juego = df.nombre_juego.drop_duplicates().to_frame().reset_index(drop=True)
juego

,nombre_juego
0,Scythe
1,Betrayal at House on the Hill
2,Catan
3,Agricola
4,Arkham Horror: The Card Game
5,Codenames
6,Memoir '44
7,Pandemic
8,Dominion
9,Ajedrez


#### PERSONAL

In [11]:
personal = df.personal.drop_duplicates().to_frame(name='nombre_personal')
personal

,nombre_personal
0,Alberto Torralba Sierra
1,Joel Ángel Campillo
2,Julio Luján Valderrama
3,Amado del Piñeiro
4,Débora Escolano Martinez
...,...
49995,Nélida Bauzà Porras
49996,Régulo del Coca
49997,Rosalva Fuente-Herrera
49998,Jose Luis Reig


#### COPIA_JUEGO

In [12]:
copia_juego = df[['id_juego','nombre_juego', 'coste_juego']].drop_duplicates().reset_index(drop=True)
copia_juego

,id_juego,nombre_juego,coste_juego
0,10,Scythe,60
1,8,Betrayal at House on the Hill,35
2,8,Catan,45
3,2,Agricola,55
4,4,Arkham Horror: The Card Game,55
5,9,Codenames,25
6,1,Memoir '44,50
7,4,Pandemic,50
8,6,Dominion,45
9,6,Ajedrez,5


#### PARTIDA

In [13]:
partida = df[['id_partida','id_juego', 'nombre_juego']].drop_duplicates()
partida

,id_partida,id_juego,nombre_juego
0,1,10,Scythe
1,2,8,Betrayal at House on the Hill
2,3,8,Catan
3,4,2,Agricola
4,5,4,Arkham Horror: The Card Game
...,...,...,...
49995,49996,2,Agricola
49996,49997,4,Pandemic
49997,49998,8,Ticket to Ride
49998,49999,1,Terraforming Mars


#### EXPANSION

In [14]:
expansion = df[['nombre_expansion','nombre_juego']].drop_duplicates().dropna().reset_index(drop=True)
expansion

,nombre_expansion,nombre_juego
0,Invaders from Afar,Scythe
1,Ciudades y Caballeros,Catan
2,Navegadores,Catan
3,Imágenes,Codenames
4,Duet,Codenames
5,Campaigns,Memoir '44
6,In the Lab,Pandemic
7,Estado de Emergencia,Pandemic
8,Hinterlands,Dominion
9,Guilds,Dominion


#### COPIA_EXPANSION

In [15]:
copia_expansion = df[['id_expansion', 'nombre_expansion', 'coste_expansion']].drop_duplicates().dropna().reset_index(drop=True)
copia_expansion

,id_expansion,nombre_expansion,coste_expansion
0,4,Invaders from Afar,30
1,9,Ciudades y Caballeros,30
2,8,Navegadores,25
3,8,Imágenes,13
4,3,Duet,12
5,3,Campaigns,18
6,10,In the Lab,25
7,7,Estado de Emergencia,27
8,3,Hinterlands,17
9,8,Guilds,19


#### UTILIZA_EXPANSION

In [16]:
utiliza_expansion = df[['id_partida','id_expansion','nombre_expansion']].drop_duplicates().dropna()
utiliza_expansion

,id_partida,id_expansion,nombre_expansion
0,1,4,Invaders from Afar
2,3,9,Ciudades y Caballeros
2,3,8,Navegadores
5,6,8,Imágenes
5,6,3,Duet
...,...,...,...
49987,49988,6,Turmoil
49991,49992,10,In the Lab
49991,49992,10,On the Brink
49995,49996,4,Farmers of the Moor


#### JUEGA

In [17]:
juega = df[['nombre_jugador','id_partida']].drop_duplicates()
juega

,nombre_jugador,id_partida
0,Melisa de Barrera,1
0,Toño Arroyo Vilalta,1
0,María Del Carmen Caballero Caparrós,1
0,Ariadna Calvo,1
0,Moisés Álamo-Llano,1
...,...,...
49999,Cesar Escribano Adán,50000
49999,Rubén Conesa Arcos,50000
49999,Jennifer Bello Casares,50000
49999,Jacinta Castilla Amaya,50000


#### GANA

In [18]:
gana = df[['resultado','id_partida']].drop_duplicates()
gana

,resultado,id_partida
0,María Del Carmen Caballero Caparrós,1
1,René Valentín Catalá,2
2,Haroldo Saez-Blanca,3
3,Juan Manuel Oller Palomo,4
4,Alonso Victor Anaya Borrell,5
...,...,...
49995,Íñigo Badía Talavera,49996
49996,Gerónimo Expósito Carreras,49997
49997,Alondra Lara Llamas,49998
49998,Gertrudis de Manrique,49999


#### ORGANIZA

In [19]:
organiza = df[['personal','id_partida', 'fecha','hora']].drop_duplicates()
organiza = df[['personal','id_partida', 'fecha','hora']].drop_duplicates()
organiza['hora'] = pd.to_datetime(organiza['hora'], format='%H:%M').dt.time


In [20]:
copia_juego = copia_juego.astype({'id_juego': int, 'coste_juego': float})
copia_expansion = copia_expansion.astype({'id_expansion': int, 'coste_expansion': int})
utiliza_expansion = utiliza_expansion.astype({'id_partida': int, 'id_expansion': int})
organiza

,personal,id_partida,fecha,hora
0,Alberto Torralba Sierra,1,2021-08-24,14:44:00
1,Joel Ángel Campillo,2,2023-07-02,18:43:00
2,Julio Luján Valderrama,3,2021-04-06,14:46:00
3,Amado del Piñeiro,4,2020-10-02,02:20:00
4,Débora Escolano Martinez,5,2024-01-30,14:09:00
...,...,...,...,...
49995,Nélida Bauzà Porras,49996,2022-02-19,14:25:00
49996,Régulo del Coca,49997,2024-10-15,05:18:00
49997,Rosalva Fuente-Herrera,49998,2024-09-16,13:36:00
49998,Jose Luis Reig,49999,2020-05-02,08:48:00


In [21]:
gana.rename(columns={'resultado': 'nombre_jugador'}, inplace=True)
organiza.rename(columns={'personal': 'nombre_personal'}, inplace=True)

In [22]:
jugador.to_csv('csvs/jugador.csv', index=False, sep=';')
juego.to_csv('csvs/juego.csv', index=False, sep=';')
personal.to_csv('csvs/personal.csv', index=False, sep=';')
copia_juego.to_csv('csvs/copia_juego.csv', index=False, sep=';')
partida.to_csv('csvs/partida.csv', index=False, sep=';')
expansion.to_csv('csvs/expansion.csv', index=False, sep=';')
copia_expansion.to_csv('csvs/copia_expansion.csv', index=False, sep=';')
utiliza_expansion.to_csv('csvs/utiliza_expansion.csv', index=False, sep=';')
juega.to_csv('csvs/juega.csv', index=False, sep=';')
gana.to_csv('csvs/gana.csv', index=False, sep=';')
organiza.to_csv('csvs/organiza.csv', index=False, sep=';')


# PASO A SQL

In [23]:
# with open('db_pass.txt', 'r', encoding='utf8') as f:
#     DB_PASS = f.read().strip()
    
# DB_PASS

In [24]:
from sqlalchemy import create_engine
import os
import sys

# Configuración
DB_USER = 'root'
DB_PASS = ''  # Cambia esto a tu contraseña real
DB_HOST = 'localhost'
DB_NAME = 'trabajo'
CSV_DIR = 'csvs'

# Crear Engine de SQLAlchemy
connection_string = f'mysql+mysqlconnector://{DB_USER}:{DB_PASS}@{DB_HOST}/{DB_NAME}'
try:
    engine = create_engine(connection_string, pool_recycle=3600)
    with engine.connect() as connection:
        print("Engine de SQLAlchemy creado y conexión probada exitosamente.")
except Exception as e:
    print(f"Error fatal creando engine de SQLAlchemy o conectando: {e}")
    sys.exit("No se puede continuar sin conexión a la base de datos.")
    
# Orden de Carga
ORDEN_CARGA = [
    ('jugador.csv',              'jugador'),
    ('juego.csv',                'juego'),
    ('personal.csv',             'personal'),
    ('expansion.csv',            'expansion'),
    ('copia_juego.csv',          'copia_juego'),
    ('copia_expansion.csv',      'copia_expansion'),
    ('partida.csv',              'partida'),
    ('juega.csv',                'juega'),
    ('gana.csv',                 'gana'),
    ('organiza.csv',             'organiza'),
    ('utiliza_expansion.csv',    'utiliza')
]

# Bucle de Carga
print("\nIniciando carga de datos")
total_filas_procesadas = 0

for csv_file, table_name in ORDEN_CARGA:
    filepath = os.path.join(CSV_DIR, csv_file)
    print(f"--- Procesando {csv_file} para tabla {table_name} ---")
    try:
        df = pd.read_csv(filepath, sep=';', keep_default_na=False, na_values=[''], engine='python')

        if df.empty:
            print(f"Archivo {csv_file} está vacío. Saltando.")
            continue

        df.to_sql(
            name=table_name,
            con=engine,
            if_exists='append',
            index=False,
            chunksize=1000,
            method=None
        )
        print(f"Datos de {csv_file} cargados (o intento de carga) en tabla {table_name}. Filas en DataFrame: {len(df)}.")
        total_filas_procesadas += len(df)
    except Exception as e:
        print(f"  ERROR procesando {filepath}: {e}")
        print(f"  Revisa los datos en {csv_file} y la estructura de la tabla {table_name}.")

print(f"\nCarga completada. Total de filas procesadas en todos los DataFrames: {total_filas_procesadas}")

if 'engine' in locals() and engine:
    engine.dispose()
    print("Engine de SQLAlchemy eliminado.")

Error fatal creando engine de SQLAlchemy o conectando: No module named 'mysql'


SystemExit: No se puede continuar sin conexión a la base de datos.

c:\Users\MAM\Documents\GitHub\bbdd_mja\bbddenv\Lib\site-packages\IPython\core\interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
